In [1]:
%load_ext autoreload
%autoreload 2

from autojobs import setup_impact
import numpy as np
from templates import TMPL

In [4]:
# place where you want to store all you simulations, should be a folder in the work scratch not login scratch.
# For example: saveloc = '~/work/superearth/impact/
saveloc='.' 

# location of the parameter file, this is a fake parameter file, you need to change it to your own parameter file in the toolkit.
parameter_loc = './fake_parameters_file.yml'

# location of target and impactor snapshots, again, you need to change it to your own snapshots.
loc_tar='./target_snap.hdf5'
loc_imp='./target_snap.hdf5'

# Here I am submitting 4 jobs, 2 different impact parameters and 2 different impact velocities (1 and 2 time mutual escape velocity in km/s !!!).
# Be careful, the impact velocity is in km/s!!!
# Provide tmpl with the templates stored in the templates.py.
# I have slightly differnt templates (TMPL) for different HPCs, you can find them in the templates.py file. You need modify it in order to run on your hpc:
# * Account name to your own account name
# * The location to your yq, for example my is "~/yq/yq_linux_386"
# * The location where you store your SWIFT executable, for example my is "/user/home/qb20321/hammer/toolkit/{swift_exe}"

# For regular impact, feep mX ,mZ, and ifspin as False.
# Most of time, you need to control:
# ncpu: number of cpus you want to use. 28 on BC and 24, 28 or 32 on BP.
# swift_exe: the name of your swift executable, for example, mine is "swift_subtask", just the name of your executable, not the location.
# time_end: the total time you want to run, in seconds. Here 108000s = 30 hours.
# delta_time: the time interval you want to save the snapshots, in seconds. Here 36000s = 10 hours, output a snapshot every 10 hours.
# time and mem: the time and memory you want to request for each job, here 24 hours and 100G.
# test: If you want to automatically submit the job, set test=0, if you just want to just generate the bash script, set test=1.
# threads: number of threads you want to use, normally the same as ncpu.
for bb in [0.4,0.7]:
    for vv in np.array([1.,2.]) * 15.36:
        setup_impact(loc_tar,loc_imp,save_folder=saveloc,tmpl=TMPL,b_value=bb,v_value=vv,mX=False,mZ=False,parameter_loc=parameter_loc,ifspin=False,
                     period=0.0,partition_name="compute",ncpu=32,hmax=0.2,exclude='[227-228]',dtmax=5.0,CFL=0.2,swift_exe='swift_subtask',
                     time_end=108000,delta_time=36000,mem='100G',time='0-24:00:00',test=1,threads=32,chmod=0,verbose=0)


# After run the code, you can find a folder automatically generated in your save_folder, for example, here is "saveloc/pX".
# pX is strcutred as:
# pX
# |── b0d4
# │   ├── v15d36kms
# │   │   ├── impact.sh
# │   │   ├── PLANETimpact_0d0h_0d99640_npt222272_1d99281_v15d3600kms_b0d400_pX.hdf5
# │   │   v30d72kms
# │   │   ├── impact.sh
# |   |   ├── PLANETimpact_0d0h_0d99640_npt222272_1d99281_v30d7200kms_b0d400_pX.hdf5
# |── b0d7
# ...

# So you can find the bash script and initial condition for each simulation in the corresponding folder.
# If you set test=1, you can just manully submit the bash script in each folder.
